### Import Library ###

In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import glob
from keras.preprocessing import image
from scipy import spatial

## 8.1 Prepare features

### Main images

In [2]:
# Load main image
main_image = cv2.imread('')

In [ ]:
# Define tile size
tile_size = (height, width) = main_image.shape[:2]

In [ ]:
# Create main image feature grid of subimage
  # Calculate average (mean) color of each subimage with tile_size


### Tile images

In [ ]:
# Collect filenames of all tile images
All_files = glob.glob()

In [ ]:
# Load tile images and resize to tile_size
tiles=[]
tile = load_img(target_size=tile_size)
tiles.append(tile)

In [ ]:
# Create array of average (mean) color of each tile image
Tile_features = []
for tile in tiles:
  mean_color = np.array(tile).mean(axis=(0,1))
  Tile_features.append(mean_color)

## 8.2 Image Clustering and Matching using KNN with KDTree Algorithm ##

In [ ]:
#Create KDTree of tile color features
tree = spatial.KDTree(Tile_features)

In [ ]:
#KNN (k=1) search for best matched tiles with each subimage
for i in range(height):
  for j in range(width):
    closest = tree.query(main_image_feature [i,j])
    closest_tiles[i, j] = closest[1]

## 8.3 Create Image Mosaic

In [ ]:
# Fill each subimage with matched tile
# Offset of tile
x, y = i*tile_size[0], j*tile_size[1]
# Index of tile
index = closest_tiles[i, j]
main_photo[x:(x+tile_size[0]), y:(y+tile_size[1]),:] = tiles[index]